In [1]:
from ipywidgets import interact

from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row, gridplot
from bokeh.models import Range1d

output_notebook()

Loading BokehJS ...

In [2]:
# Globals
oo = float("inf")
quad_width = 0.02
default_plot_size = (300, 300)
default_color = "black"
default_y_range = Range1d(-1.5, 1.5)

def delete_toolbar(plot):
    # remove the logo and the toolbar
    plot.toolbar.logo = None
    plot.toolbar_location = None

# Beipsiel 1

In [3]:
n = [-3, -2,   -1, 0,   1, 2, 3]
default_x_range = Range1d(n[0]-1, n[-1]+1)

# x[n]
#    [-3, -2,   -1, 0,   1, 2, 3]
xn = [0,  0, -1, 0.6, 1.2, 0, 0]
xn_p = figure(width=default_plot_size[0], height=default_plot_size[1], 
              title="x[n]", x_axis_label="n", y_axis_label="x[n]",tools=[])

# input signal
xn_color = "black"
xn_quad = xn_p.quad(top=xn, bottom=0, left=[i-quad_width for i in n], right=[i+quad_width for i in n], color=xn_color)
xn_cir = xn_p.circle(n, xn, size=10, color=xn_color, fill_color="white")

# -------------------------------------------------------------------------------
# x[k] delta[n-k]
xd = [0 for _ in n]
xd_p = figure(width=default_plot_size[0], height=default_plot_size[1], 
              title="x[k] delta[n-k]", x_axis_label="n", y_axis_label="x[k] delta[n-k]",tools=[])

# input signal
xd_color = "black"
xd_quad = xd_p.quad(top=xd, bottom=0, left=[i-quad_width for i in n], right=[i+quad_width for i in n], color=xd_color)
xd_cir = xd_p.circle(n, xd, size=10, color=xd_color, fill_color="white")

# -------------------------------------------------------------------------------
# h_k[n]
h = [0 for _ in n]
h_p = figure(width=default_plot_size[0], height=default_plot_size[1], 
              title="h_k[n]", x_axis_label="n", y_axis_label="h_k[n]",tools=[])

# input signal
h_color = "black"
h_quad = h_p.quad(top=h, bottom=0, left=[i-quad_width for i in n], right=[i+quad_width for i in n], color=h_color)
h_cir = h_p.circle(n, h, size=10, color=h_color, fill_color="white")

# -------------------------------------------------------------------------------
# x[k] h_k[n]
xh = [0 for _ in n]
xh_p = figure(width=default_plot_size[0], height=default_plot_size[1], 
              title="x[k] h_k[n]", x_axis_label="n", y_axis_label="x[k] h_k[n]",tools=[])

# input signal
xh_color = "black"
xh_quad = xh_p.quad(top=xh, bottom=0, left=[i-quad_width for i in n], right=[i+quad_width for i in n], color=xh_color)
xh_cir = xh_p.circle(n, xh, size=10, color=xh_color, fill_color="white")

# -------------------------------------------------------------------------------
# y[n]
# y = [0,0,-2.2, 0, -2.2, -3.3, 0.7]
y = [0 for _ in n]
y_p = figure(width=default_plot_size[0], height=default_plot_size[1], 
              title="y[n]", x_axis_label="n", y_axis_label="y[n]",tools=[])

# input signal
y_color = "black"
y_quad = y_p.quad(top=y, bottom=0, left=[i-quad_width for i in n], right=[i+quad_width for i in n], color=y_color)
y_cir = y_p.circle(n, y, size=10, color=y_color, fill_color="white")
# -------------------------------------------------------------------------------
all_p = [xn_p, xd_p, h_p, xh_p, y_p]
for p in all_p:
    delete_toolbar(p)
    
    # ranges
    p.x_range = default_x_range
    p.y_range = default_y_range
    
    # axes
    p.segment(x0=min(n)-1, y0=0, x1=max(n)+1, y1=0, line_width = 2,color="gray", line_alpha=0.5)    # x_axis
    p.segment(x0=0, y0=min(n)-1, x1=0, y1=max(n)+1, line_width = 2,color="gray", line_alpha=0.5)    # y_axis


# ranges
# y_p.y_range = Range1d(-2.5, 2.5)
    
# -------------------------------------------------------------------------------
# update
k_ls = []
def update(k = -3):
    new_k = len(n) // 2 + k
    
    new_xd = [0 for _ in n]
    new_xd[new_k] = xn[new_k]
    xd_quad.data_source.data['top'] = new_xd
    xd_cir.data_source.data['y'] = new_xd
    xd_p.title.text = "x[{}] delta[n-{}]".format(k, -k)
    
    if k == -1:   new_h = [0, 0, 1, 0.7, -0.3, 0.7, 0]
    elif k == 0:  new_h = [0, 0, -0.3, 1, 0.3, 0, 0.3]
    elif k == 1:  new_h = [0, 0, 0.3, 0, -1, -0.3, 0]
    else:         new_h = [0] * len(n)
    h_quad.data_source.data['top'] = new_h
    h_cir.data_source.data['y'] = new_h
    h_p.title.text = "h_{}[n]".format(k)
    
    new_xh = [xn[new_k] * new_h[i] for i in range(len(new_h))]
    xh_quad.data_source.data['top'] = new_xh
    xh_cir.data_source.data['y'] = new_xh
    xh_p.title.text = "x[{}] h_{}[n]".format(k, k)
    
    if k not in k_ls:
        k_ls.append(k)
        for i in range(len(y)):
            y[i] += new_xh[i]
        y_quad.data_source.data['top'] = y
        y_cir.data_source.data['y'] = y
    
    push_notebook()

In [4]:
grid = gridplot([[xn_p, None, y_p ],
                 [xd_p, h_p, xh_p]])

show(grid, notebook_handle=True)

In [5]:
interact(update, k=(-3,3))